# **IEOR E4650  Business Analytics (Fall 2019)**

##**Lecture 9: Poisson Model with Observed Heterogeneity**

In this lecture, we discuss how to model heterogeneity for a count model. 

Learning objective:

* Understand the idea of uobserved heterogeneity
* Understand how to estimate a count model with observed heterogeneity
* Understand how to use the model for prediction



##Observed Heterogeneity
###(A model with covariates)

For models that have the unobserved heterogeneity elements in, we were relying on the distribution of the data itself to tell us how the distribution of $\lambda$ looks. 

There are many factors that could determine the $\lambda$ for each customer. For example, the value of $\lambda$ might be affected by the gender of the customer, the age of the customer.... If we have the information in the data, we can use it to model the variations in $\lambda$ directly. This is called modeling the observed heterogeneity. 

For this lecture, we will use a count data with covariates.

Setting of this study:

The state wildlife biologists want to model how many fish are being caught by fishermen at a state park. Visitors are asked whether or not they have a camper, how many people were in the group, were there children in the group and how many fish were caught. Some visitors do not fish, but there is no data on whether a person fished or not. Some visitors who did fish did not catch any fish so there are excess zeros in the data because of the people that did not fish.



|Variable|Description|
|---|---|
|livebait|whether the visitor has live bait or not|
|child| how many children were in the party |
|persons| how many people were in the party|
|camper| whether this party camped|
|Nfish| The number of fish caught|


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link="https://drive.google.com/open?id=1ytZ6QqLH-ES1YxxYbOGwMERVep0Tkhgu"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('myfile.csv')  
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import scipy.stats as spst
from scipy.optimize import minimize
import scipy.special as spsp


Fishing = pd.read_csv('myfile.csv')
Fishing.head(10)


,livebait,child,persons,camper,Nfish
0,0,0,1,0,0
1,1,0,1,1,0
2,1,0,1,0,0
3,1,1,2,1,0
4,1,0,1,0,1
5,1,2,4,1,0
6,1,1,3,0,0
7,1,3,4,0,0
8,0,2,3,1,0
9,1,0,1,1,1


##Basic statistics

Let's use some basic statistics to check how different factors affect the number of fish caught individually

In [0]:
Fishing[["livebait","Nfish"]].groupby("livebait").mean()

,Nfish
livebait,
0,0.558824
1,3.726852


In [0]:
Fishing[["child","Nfish"]].groupby("child").mean()

,Nfish
child,
0,5.189394
1,1.760000
2,0.212121
3,0.000000


In [0]:
Fishing[["persons","Nfish"]].groupby("persons").mean()

,Nfish
persons,
1,0.736842
2,1.471429
3,2.929825
4,7.757576


In [0]:
Fishing[["camper","Nfish"]].groupby("camper").mean()

,Nfish
camper,
0,1.524272
1,4.537415


## Simple Poisson model with covarites.

Previously, we used $exp(\beta_0)$ to estimate $\lambda$.

If we have a model with covariates, we can simply use  $\lambda_i=exp(\beta_0+\beta_1x_{1i}+\beta_2x_{2i})$. In this case, every customer will have a different $\lambda$ depending on the values of the covariates.

After we construct the individualized $\lambda_i$, the rest of the work is the same.

Now, for customer $i$, the probability of getting $y_i$ counts is simply 

$$PMF_{poisson}(y_i|\lambda_i)=\frac{exp(-\lambda_i)\lambda_i^y}{y!}$$

Notice that what's differentiate a simple Poisson model with covariates from a simple Poisson distribution is that every customer will be using a different $\lambda_i$ when computing the individual likelihood.

After that, we will move on to compute the joint log likelihood like we did before. Instead of estimating $\beta_0$ only, we will estimate $\beta_0$, $\beta_1$, $\beta_2$.

In [0]:
x1=Fishing["livebait"]
x2=Fishing["child"]
x3=Fishing["persons"]
x4=Fishing["camper"]
y=Fishing["Nfish"]

def Neg_LL(betas):
  #include observed heterogeneity in lmbda
  lmbda= np.exp(betas[0]+betas[1]*x1+betas[2]*x2+betas[3]*x3+betas[4]*x4)
  Ind_L= spst.poisson.pmf(y,lmbda)
  Ind_LL=np.log(Ind_L)
  return -np.sum(Ind_LL)

import warnings
warnings.simplefilter("ignore")

guess=np.random.rand(5)
model1=minimize(Neg_LL,guess, method="BFGS")

print(model1.fun)
print(model1.x)

791.977641149574
[-3.39007447  1.66967212 -1.70993266  1.07395579  0.80009263]


In [0]:
betas=model1.x
np.exp(betas[0])

0.03370616677184225

Now, for every customer, we will have a personalized exposure.

In [0]:
betas=model1.x
#predict lambda for everyone. This is the expected number of exposures
predicted_lambda=np.exp(betas[0]+betas[1]*x1+betas[2]*x2+betas[3]*x3+betas[4]*x4)

#add the predicted lambda to the data frame.
Fishing=Fishing.assign(predicted_lambda=predicted_lambda)
Fishing.head(10)

,livebait,child,persons,camper,Nfish,predicted_lambda
0,0,0,1,0,0,0.098656
1,1,0,1,1,0,1.166078
2,1,0,1,0,0,0.523904
3,1,1,2,1,0,0.617343
4,1,0,1,0,1,0.523904
5,1,2,4,1,0,0.956617
6,1,1,3,0,0,0.811827
7,1,3,4,0,0,0.077741
8,0,2,3,1,0,0.061545
9,1,0,1,1,1,1.166078


In [0]:
#AIC -2*JLL+2*k
print(2*model1.fun+2*5)

#BIC -2*JLL+ln(n)*k
print(2*model1.fun+np.log(len(y))*5)

1593.955282299148
1611.5625868884592


In [0]:
x1=Fishing["livebait"]
x2=Fishing["child"]
x3=Fishing["persons"]
x4=Fishing["camper"]
y=Fishing["Nfish"]

def Neg_LL(betas):
  #include observed heterogeneity in lmbda
  lmbda= np.exp(betas[0])
  Ind_L= spst.poisson.pmf(y,lmbda)
  Ind_LL=np.log(Ind_L)
  return -np.sum(Ind_LL)

import warnings
warnings.simplefilter("ignore")

guess=np.random.rand(1)
model10=minimize(Neg_LL,guess, method="BFGS")

print(model10.fun)
print(model10.x)

#AIC -2*JLL+2*k
print(2*model10.fun+2*1)

#BIC -2*JLL+ln(n)*k
print(2*model10.fun+np.log(len(y))*1)

1647.7155682031048
[1.1927096]
3297.4311364062096
3300.952597324072


### Incorporating the covariates in a Mixture model

Again, recall the individual likelihood function we had before. 

$PMF_{poisson}(y_i|\lambda_1) p+ PMF_{poisson}(y_i|\lambda_2) (1-p)$

$PMF_{poisson}(y_i|\lambda_1)$ is the PMF of a Poisson distribution with $\lambda_1$ at $y_i$. Similarly, $PMF_{poisson}(y_i|\lambda_2)$ is the PMF of a Poisson distribution with $\lambda_2$ at $y_i$. 



So here we have three options:

(1) Use the covarites to model $\lambda_1$ using $exp(\beta_0+\beta_1 x_1+\beta_2 x_2)$

(2) Use the covariates to model $\lambda_2$ using $exp(\beta_0+\beta_1 x_1+\beta_2 x_2)$

(3) Use the covariates to model p using using $\frac{exp(\beta_0+\beta_1 x_1+\beta_2 x_2)}{exp(\beta_0+\beta_1 x_1+\beta_2 x_2)+1}$

Of course, you can also incorperate the covariates in two of the parameters or all three of them. However, this will mean possible overfit. The model might fail to converge altogether.





In [0]:
#Let's estimate a zero-inflated Poisson model. We incorporate both the covariates in 
#lambda1 and p
def Neg_LL(betas):
  lmbda= np.exp(betas[0]+betas[1]*x1+betas[2]*x2+betas[3]*x3+betas[4]*x4)
  exp_p=np.exp(betas[5]+betas[6]*x1+betas[7]*x2+betas[8]*x3+betas[9]*x4)
  p= exp_p/(1+exp_p)
  Ind_L1=spst.poisson.pmf(y,lmbda)
  Ind_L2=spst.poisson.pmf(y,0)
  Ind_L=Ind_L1*p+Ind_L2*(1-p)
  Ind_LL=np.log(Ind_L)
  return -np.sum(Ind_LL)



In [0]:
guess=np.random.rand(10)
model2=minimize(Neg_LL,guess, method="BFGS")
 
print(model2.fun)
print(model2.x)
betas=model2.x

712.4108904555123
[-2.45200151  1.81881386 -1.18063817  0.83635841  0.59706376 -0.8337119
 -0.74080075 -1.95857734  0.93116163  0.87166932]


###Posterior Analysis

Because we have unoberved heterogeneity, we can again conduct posterior analysis. If we see a customer for one period, we can use the outcome to update the probability of this customer belonging to each segment, which can then help us update the expected number of exposures for this customer if we see this customer again.

In [0]:
#livebait=1, child=2, persons=3, camper=1
#don't know Nfish
#without posterior analysis
x1s=1
x2s=2
x3s=3
x4s=1

lmbda= np.exp(betas[0]+betas[1]*x1s+betas[2]*x2s+betas[3]*x3s+betas[4]*x4s)
exp_p=np.exp(betas[5]+betas[6]*x1s+betas[7]*x2s+betas[8]*x3s+betas[9]*x4s)
p= exp_p/(1+exp_p)
#expected number of fish caught if this person went fishing
#the probability this person went fishing
print(lmbda,p)
#expected number of fish caught unconditional on whether this person went fishing
print(lmbda*p+0*(1-p))

1.1181494175566526 0.13865418461583437
0.15503609576998778


In [0]:
#livebait=1, child=2, persons=3, camper=1
# Nfish=4
#posterior analysis
x1s=1
x2s=2
x3s=3
x4s=1
ys=4

lmbda= np.exp(betas[0]+betas[1]*x1s+betas[2]*x2s+betas[3]*x3s+betas[4]*x4s)
exp_p=np.exp(betas[5]+betas[6]*x1s+betas[7]*x2s+betas[8]*x3s+betas[9]*x4s)
p= exp_p/(1+exp_p)
Ind_L1=spst.poisson.pmf(ys,lmbda)
Ind_L2=spst.poisson.pmf(ys,0)
Ind_L=Ind_L1*p+Ind_L2*(1-p)
p_post=Ind_L1*p/Ind_L
print(p_post)

#expected number of fish caught unconditional on whether this person went fishing
print(lmbda*p_post+0*(1-p_post))

1.0
1.1181494175566526


##Incorporating the covariates in an NBD model

For a simple Poisson model, we assumed that 

$\lambda_i=exp(\beta_0+ \beta_1 x_{1i}+ \beta_2 x_{2i})=\lambda_0exp(\beta_1 x_{1i}+ \beta_2 x_{2i})$.

It might make sense to assume that people have different $\lambda_0$ values. Again, let's assume $\lambda_0 \sim  Gamma(\gamma, \alpha)$. 

$\lambda_0 \sim  Gamma(\gamma, \alpha)$
We will have 

$\lambda_i=\lambda_0exp(\beta_1 x_{1i}+ \beta_2 x_{2i})\sim Gamma (\gamma, \frac{\alpha}{exp(\beta_1x_1+\beta_2 x_2)})$. 

Thus, we will have the individual likelihood function for NBD with covariates equals to:

$\frac{\Gamma(\gamma+y)}{\Gamma(\gamma)\Gamma(y+1)}(\frac{exp(\beta_1 x_1 + \beta_2 x_2 )}{\alpha+exp(\beta_1 x_1 + \beta_2 x_2 )})^y(\frac{\alpha}{\alpha+exp(\beta_1 x_1 + \beta_2 x_2)})^\gamma$

For this distribution $E(x)=\frac{\gamma }{\alpha}exp(\beta_1 x_1 + \beta_2 x_2)$


In [0]:
def Neg_LL(betas):
  alpha= np.exp(betas[0])
  exp_part=np.exp(betas[1]*x1+betas[2]*x2+betas[3]*x3+betas[4]*x4)
  gamma= np.exp(betas[5])
  ind_L_part1= spsp.gamma(gamma+y)/spsp.gamma(gamma)/spsp.gamma(y+1)
  ind_L_part2=(exp_part/(alpha+exp_part))**y*(alpha/(alpha+exp_part))**gamma
  ind_L=ind_L_part1*ind_L_part2
  #individual log likelihoo
  ind_LL=np.log(ind_L)
  #joint log likelihood
  Joint_LL=np.sum(ind_LL)
  return -Joint_LL  
  
 

In [0]:
guess=np.random.rand(6)
result=minimize(Neg_LL,guess,method="BFGS")
betas=result.x
 

print(betas)
print(result.fun)
alpha= np.exp(betas[0])
exp_part=np.exp(betas[1]*x1+betas[2]*x2+betas[3]*x3+betas[4]*x4)
gamma= np.exp(betas[5])
lambda_predict_nbd=gamma/alpha*exp_part
Fishing=Fishing.assign(lambda_predict_nbd=lambda_predict_nbd)
Fishing.head(10)


[ 2.31339355  1.53793921 -1.80094181  1.06176219  0.51786184 -0.66260676]
398.58071478200293


,livebait,child,persons,camper,Nfish,lambda_predict_nbd
0,0,0,1,0,0,0.147454
1,1,0,1,1,0,1.152073
2,1,0,1,0,0,0.686397
3,1,1,2,1,0,0.550121
4,1,0,1,0,1,0.686397
5,1,2,4,1,0,0.759546
6,1,1,3,0,0,0.947701
7,1,3,4,0,0,0.074733
8,0,2,3,1,0,0.056431
9,1,0,1,1,1,1.152073


###Posterior Analysis

Previously, we have 

$P(y|\lambda) \sim Poisson$ and $\lambda\sim Gamma (\gamma,\alpha)$. We derived that the posterior distribution of $\lambda|y \sim Gamma(\gamma+y, \alpha+1)$.

When incorprating the covariates, we will have 

$P(y|\lambda) \sim Poisson$ and $\lambda\sim Gamma (\gamma,\frac{\alpha}{exp(\beta_1x_1+\beta_2x_2)})$. It is easy to see that the posterior distribution of $\lambda|y \sim Gamma(\gamma+y, \frac{\alpha}{exp(\beta_1x_1+\beta_2x_2)}+1)$.

Thus, the expected number of exposures for a customer conditional on observing $y$ is equal to 

$$\frac{\gamma+y}{\frac{\alpha}{exp(\beta_1x_1+\beta_2x_2)}+1}=exp(\beta_1x_1+\beta_2x_2)\frac{\gamma+y}{\alpha+exp(\beta_1x_1+\beta_2x_2)}$$


In [0]:
lambda_posterior=exp_part*(gamma+y)/(alpha+exp_part)
Fishing.assign(lambda_posterior=lambda_posterior)

,livebait,child,persons,camper,Nfish,lambda_predict_nbd,lambda_posterior
0,0,0,1,0,0,0.147454,0.114658
1,1,0,1,1,0,1.152073,0.356145
2,1,0,1,0,0,0.686397,0.294401
3,1,1,2,1,0,0.550121,0.266126
4,1,0,1,0,1,0.686397,0.865493
5,1,2,4,1,0,0.759546,0.307086
6,1,1,3,0,0,0.947701,0.333887
7,1,3,4,0,0,0.074733,0.065270
8,0,2,3,1,0,0.056431,0.050863
9,1,0,1,1,1,1.152073,1.047011
